# Bachelor Programmierkurs 2016
In diesem Notebook sollen die grundlegenden Konzepte von RooFit vermittelt werden.

In [20]:
import ROOT

import os

import matplotlib.pyplot
%matplotlib inline

Erstelle ein Fitmodell, bestehend aus einer Signal- (Gaußverteilung) und einer Untergrundkomponente (Argus, sieht etwa [so](https://inspirehep.net/record/833350/files/figs_0140804-020.png) aus)

In [21]:
# Observable
mass = ROOT.RooRealVar('mass', 'mass', 5000, 5500, 'GeV / #textit{c}^2')

# Erstelle eine Signal-PDF, bestehend aus einer Gaußfunktion
sigMean = ROOT.RooRealVar('sigMean', 'sigMean', 5280, 5200, 5300)
sigWidth = ROOT.RooRealVar('sigWidth', 'sigWidth', 20, 1, 100)
sigGauss = ROOT.RooGaussian('sigGauss', 'sigGauss PDF', mass, sigMean, sigWidth)

# Erstelle eine Untergrund-PDF, bestehend aus einer Argusfunktion
bkgArgPar = ROOT.RooRealVar('bkgArgPar', 'bkgArgPar', -20, -100, -1)
bkgArgConst = ROOT.RooRealVar('bkgArgConst', 'bkgArgConst', 5291)
bkgArgus = ROOT.RooArgusBG('bkgArgus', 'bkgArgus PDF', mass, bkgArgConst, bkgArgPar)

# Kombiniere Signal + Untergrung PDF
nSig = ROOT.RooRealVar('nSig', '#signal events', 1000, 200, 10000)
nBkg = ROOT.RooRealVar('nBkg', '#background events', 1000, 200, 10000)
model = ROOT.RooAddPdf('model', 'complete fitmodel', ROOT.RooArgList(sigGauss, bkgArgus), 
                                                     ROOT.RooArgList(nSig, nBkg))
model.Print('V') 

--- RooAbsArg ---
  Value State: DIRTY
  Shape State: DIRTY
  Attributes: 
  Address: 0x7fa0bdbdd800
  Clients: 
  Servers: 
    (0x7fa0bdbb9600,V-) RooGaussian::sigGauss "sigGauss PDF"
    (0x7fa0bf46cdf0,V-) RooRealVar::nSig "#signal events"
    (0x7fa0bdbe5800,V-) RooArgusBG::bkgArgus "bkgArgus PDF"
    (0x7fa0bf46d210,V-) RooRealVar::nBkg "#background events"
  Proxies: 
    !refCoefNorm -> 
    !pdfs -> 
      1)  sigGauss
      2)  bkgArgus
    !coefficients -> 
      1)  nSig
      2)  nBkg
--- RooAbsReal ---

  Plot label is "model"
--- RooAbsPdf ---
Cached value = 0


In [22]:
numDataPoints = 5000
plotPath = 'Plots'

if not os.path.exists(plotPath):
    os.makedirs(plotPath)

# Generiere ein ToyMC aus der gegebenen PDF
data = model.generate(mass, numDataPoints)

# Führe einen Fit an die Daten durch
model.fitTo(data, ROOT.Extended())

# Plotte die Datenpunkte und die PDF in ein Frame
massFrame = mass.frame()

data.plotOn(massFrame)
model.plotOn(massFrame)
model.plotOn(massFrame, ROOT.Components(argus), ROOT.LineStyle(ROOT.kDashed))

# Erzeuge ein TCanvas und plotte das Frame.
can = ROOT.TCanvas('can', 'can', 800, 600)

massFrame.Draw()

can.Draw()

TypeError: none of the 5 overloaded methods succeeded. Full details:
  RooDataSet* RooAbsPdf::generate(const RooArgSet& whatVars, const RooDataSet& prototype, int nEvents = 0, bool verbose = kFALSE, bool randProtoOrder = kFALSE, bool resampleProto = kFALSE) =>
    could not convert argument 1
  RooDataSet* RooAbsPdf::generate(const RooArgSet& whatVars, int nEvents, const RooCmdArg& arg1, const RooCmdArg& arg2 = RooCmdArg::none(), const RooCmdArg& arg3 = RooCmdArg::none(), const RooCmdArg& arg4 = RooCmdArg::none(), const RooCmdArg& arg5 = RooCmdArg::none()) =>
    takes at least 3 arguments (2 given)
  RooDataSet* RooAbsPdf::generate(const RooArgSet& whatVars, const RooCmdArg& arg1 = RooCmdArg::none(), const RooCmdArg& arg2 = RooCmdArg::none(), const RooCmdArg& arg3 = RooCmdArg::none(), const RooCmdArg& arg4 = RooCmdArg::none(), const RooCmdArg& arg5 = RooCmdArg::none(), const RooCmdArg& arg6 = RooCmdArg::none()) =>
    could not convert argument 1
  RooDataSet* RooAbsPdf::generate(RooA